In [1]:
import os
os.getcwd()

'/teamspace/studios/this_studio/text-summarizer/research'

In [2]:
os.chdir('../')
os.getcwd()

'/teamspace/studios/this_studio/text-summarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    output_dir: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    logging_dir: Path
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [4]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        os.makedirs(self.config.artifact_root, exist_ok=True)
        
        
    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        
        os.makedirs(config.root_dir, exist_ok=True)
        
        return ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            output_dir = params.output_dir,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            logging_dir = config.logging_dir,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

In [6]:
!pip install --upgrade pip
!pip uninstall torchvision -qy
!pip install torchvision -q
!pip install -U transformers -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textsummarizer 1.0.0 requires torch==2.5, but you have torch 2.6.0 which is incompatible.


In [7]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from datasets import load_from_disk

2.6.0+cu124
0.21.0+cu124


In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def start_training(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        
        model = AutoModelForSeq2SeqLM.from_pretrained(config.model_ckpt)
        tokenizer = AutoTokenizer.from_pretrained(config.model_ckpt)
        data_collator = DataCollatorForSeq2Seq(tokenizer=self.tokenizer, padding=True)
        
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        training_args = TrainingArguments(
            output_dir = self.config.output_dir,
            num_train_epochs = self.config.num_train_epochs,
            per_device_train_batch_size = self.config.per_device_train_batch_size,
            per_device_eval_batch_size = self.config.per_device_eval_batch_size,
            logging_dir = self.config.logging_dir,
            logging_steps = self.config.logging_steps,
            evaluation_strategy = self.config.evaluation_strategy,
            eval_steps = self.config.eval_steps,
            save_steps = self.config.save_steps,
            gradient_accumulation_steps = self.config.gradient_accumulation_steps,
            warmup_steps = self.config.warmup_steps,
            weight_decay = self.config.weight_decay
        )
        
        trainer = Trainer(
            model = model,
            args = training_args,
            data_collator = data_collator,
            train_dataset = dataset_samsum_pt['test'],
            eval_dataset = dataset_samsum_pt['validation']
        )
        
        trainer.train()
        
        model.save_pretrained(os.path.join(self.config.root_dir, "trained-pegausus-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "trained-pegausus-tokenizer"))

In [ ]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(model_trainer_config)
model-trainer.start_training()